In [6]:
import pyspark
from pyspark.sql import SparkSession #Necessary for initializing pyspark
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [2]:
# Creating a SparkSession
spark = (SparkSession.builder.appName("SparkSQLExample").getOrCreate())

# Path to data set
csv_file = "superstore.csv"

df = spark.read \
    .format("csv") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load(csv_file)


# Creating a temporary view to run SQL Queries
df.createOrReplaceTempView("u") 

In [15]:
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

#### Determine the best-selling product sub-category.

In [8]:
best_selling_sub_category = df.groupBy("Sub-Category").agg(F.round(F.sum("Quantity")).alias("Total Quantity Sold"))
best_selling_sub_category.orderBy("Total Quantity Sold", ascending=False).show()
# We use `F` as a shorthand for `pyspark.sql.functions` to make the code cleaner and easier to read by reducing repetitive typing.

+------------+-------------------+
|Sub-Category|Total Quantity Sold|
+------------+-------------------+
|     Storage|            10105.0|
| Furnishings|             9644.0|
|     Binders|             8576.0|
|       Paper|             7813.0|
|      Phones|             3508.0|
|         Art|             3000.0|
| Accessories|             2976.0|
|      Chairs|             2356.0|
|   Envelopes|             2017.0|
|  Appliances|             1729.0|
|      Labels|             1400.0|
|    Supplies|             1311.0|
|      Tables|             1241.0|
|   Fasteners|              917.0|
|   Bookcases|              868.0|
|    Machines|              440.0|
|     Copiers|              234.0|
+------------+-------------------+



In [9]:
best_selling_sub_category.orderBy("Total Quantity Sold", ascending=False).limit(1).show()

+------------+-------------------+
|Sub-Category|Total Quantity Sold|
+------------+-------------------+
|     Storage|            10105.0|
+------------+-------------------+



#### Identify the product category generating the highest revenue.

In [12]:
highest_rev_generating_category = df.groupBy("Category").agg(F.round(F.sum("Sales"),2).alias("Total Revenue Generated by Category"))
highest_rev_generating_category.orderBy("Total Revenue Generated by Category", ascending=False).show()

+---------------+-----------------------------------+
|       Category|Total Revenue Generated by Category|
+---------------+-----------------------------------+
|     Technology|                          835900.07|
|      Furniture|                          733046.86|
|Office Supplies|                          703502.93|
+---------------+-----------------------------------+



In [13]:
highest_rev_generating_category.orderBy("Total Revenue Generated by Category", ascending=False).limit(1).show()

+----------+-----------------------------------+
|  Category|Total Revenue Generated by Category|
+----------+-----------------------------------+
|Technology|                          835900.07|
+----------+-----------------------------------+



#### Compile a top 10 list of the most valuable customers.

In [21]:
most_valuable_customer = (
    df.groupBy("Customer Name")
    .agg(F.round(F.sum("Profit"), 2).alias("Total Profit from Customer"))
    .orderBy("Total Profit from Customer", ascending=False)
    .limit(10)
)

most_valuable_customer.show()

+--------------------+--------------------------+
|       Customer Name|Total Profit from Customer|
+--------------------+--------------------------+
|        Tamara Chand|                   8964.48|
|        Raymond Buch|                    6976.1|
|        Sanjit Chand|                   5757.41|
|        Hunter Lopez|                   5622.43|
|       Adrian Barton|                   5438.91|
|        Tom Ashbrook|                   4703.79|
|Christopher Martinez|                   3899.89|
|       Keith Dawkins|                   3038.63|
|         Andy Reiter|                   2884.62|
|       Daniel Raglin|                   2869.08|
+--------------------+--------------------------+



#### Determine the state responsible for the highest number of orders.

In [23]:
highest_ordering_state = (
    df.groupBy("State")
    .agg(F.round(F.sum("Quantity"), 2).alias("Total Orders By State"))
    .orderBy("Total Orders By State", ascending=False)
    .limit(10)
)

highest_ordering_state.show()

+------------+---------------------+
|       State|Total Orders By State|
+------------+---------------------+
|  California|             13637.27|
|    New York|               5116.2|
|       Texas|              4272.41|
|Pennsylvania|              3614.38|
|  Washington|              3541.89|
|    Illinois|              2903.23|
|        Ohio|              2863.19|
|     Florida|              1939.53|
|  New Jersey|              1602.74|
|     Arizona|              1596.94|
+------------+---------------------+



#### Find the year with the highest revenue generation.

In [46]:
# This query converts the "Order Date" column from a string format (M/d/yyyy) to a date type and creates a new column "Year" that extracts the year from the converted date.
df_with_year = df.withColumn("Order Date", F.to_date("Order Date", "M/d/yyyy")) \
                 .withColumn("Year", F.year("Order Date"))

In [47]:
most_rev_generating_year = (
    df_with_year.groupBy("Year")
    .agg(F.round(F.sum("Sales"), 2).alias("Total Sales by Year"))
    .orderBy("Total Sales by Year", ascending=False)
)

In [48]:
most_rev_generating_year.show()

+----+-------------------+
|Year|Total Sales by Year|
+----+-------------------+
|2017|          724994.56|
|2016|          601265.26|
|2014|           481763.8|
|2015|          464426.24|
+----+-------------------+

